In [1]:
import os, sys
sys.path.insert(0, '/Users/shashank/Workspace/Orgs/Ether/ai-engine/services/recommendations')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import json as js
from scipy.spatial.distance import cosine

In [4]:
from vectorize import Vectorizer
from lsh import Table, LSH, Search, MinHash
from graph_query import DgraphClient

In [13]:
vectorizer = Vectorizer()
dg = DgraphClient(url=url)

In [5]:
url = "111.93.155.194:9080"

In [6]:
def to_json(data, filename):
    with open(filename + ".json", "w", encoding="utf-8") as f_:
        js.dump(data, f_, ensure_ascii=False, indent=4)

def read_json(json_file):
    with open(json_file) as f_:
        meeting = js.load(f_)
    return meeting

In [27]:
user_kw_query = """
query mlChannelUserKw($n: string, $t: int) {
    MLchannelShashankKw(func: type("Channel"))
  @filter(eq(name, "ml-ai")) @cascade{
        uid
        xid
    	hasContext {
          xid
          attribute
          associatedMind {
            name
            type
          }
          hasMeeting (first: $t){
            xid
            hasSegment {
              authoredBy @filter(anyofterms(name, $n)) {
                name
                xid
              }
              hasKeywords {
                values
              }
            }
          }
        }
   }
}
"""

In [28]:
variables = {"$n": "shashank@etherlabs.io", "$t": "60"}
user_kw_resp = dg.perform_query(query=user_kw_query, variables=variables)

In [29]:
user_kw_resp

{'MLchannelShashankKw': [{'uid': '0x403c3',
   'xid': '898068b8-3c18-44c0-9aa0-f6135edd8f44',
   'hasContext': {'xid': '01DBB3SN99AVJ8ZWJDQ57X9TGX',
    'attribute': 'contextId',
    'associatedMind': [{'name': 'AI', 'type': 'domain'}],
    'hasMeeting': [{'xid': 'fdabb4c7-78ad-4ccd-af7b-17b89ce46534',
      'hasSegment': [{'authoredBy': {'name': 'Shashank',
         'xid': '7e7ccbba-232d-411a-a95a-d3f244a35f40'},
        'hasKeywords': {'values': ['fall back on segment wise',
          'domains groups',
          'algorithmic side',
          'mechanism or remove',
          'edge case percentage']}},
       {'authoredBy': {'name': 'Shashank',
         'xid': '7e7ccbba-232d-411a-a95a-d3f244a35f40'},
        'hasKeywords': {'values': ['noisy and stuff',
          'Keys',
          'Keys his level',
          'problem is like meet call entities',
          'kind of intersection',
          'candida graphs']}},
       {'authoredBy': {'name': 'Shashank',
         'xid': '7e7ccbba-232d-411

In [ ]:
def format_response(resp):
    user_id_dict = {}
    user_kw_dict = {}
    for info in resp["userKw"]:
        try:
            user_id = info["xid"]
            user_name = info["name"]

            user_id_dict.update({
                user_id: user_name
            })

            keyword_object = info["~authoredBy"]
            user_kw_list = []
            for obj in keyword_object:
                kw = obj["hasKeywords"]
                user_kw_list.extend([words for words in kw[0]["values"]])

            user_kw_dict.update({
                user_id: user_kw_list
            })
        except Exception:
            continue
    
    return user_kw_dict, user_id_dict

In [ ]:
user_kw_dict, user_id_dict = format_response(resp=user_kw_resp)

In [ ]:
user_kw_dict

In [ ]:
ref_user_kw_query = """
query userKw($t: int) {
    userKw(func: type("User")) {
        uid
        xid
        name
        mentionName
        ~hasMember{
            name
        }
        ~authoredBy (first: $t) {
            hasKeywords {
                values
            }
        }
    }
}
"""

In [ ]:
var = {"$t": "5"}
ref_user_kw_resp = perform_query(query=ref_user_kw_query, variables=var)

In [ ]:
ref_user_kw_resp

In [ ]:
ref_user_kw_dict, ref_user_id_dict = format_response(resp=ref_user_kw_resp)

In [ ]:
len(ref_user_id_dict.keys())

In [ ]:
len(ref_user_kw_dict)

In [ ]:
ref_user_kw_dict

In [ ]:
modified_ref_user_id_dict = {k: v for k, v in ref_user_id_dict.items() if k in ref_user_kw_dict.keys()}

In [ ]:
len(modified_ref_user_id_dict)

## Remove users (if any) that are being compared against

In [ ]:
for k in modified_ref_user_id_dict.keys():
    if k in user_kw_dict.keys():
        del ref_user_kw_dict[k]

In [ ]:
len(ref_user_kw_dict)

In [ ]:
aws_config = Config(
        connect_timeout=180,
        read_timeout=300,
        retries={"max_attempts": 2},
        region_name="us-east-1",
    )
lambda_client = bclient("lambda", config=aws_config)
lambda_function = "sentence-encoder-lambda"

def get_embeddings(input_list):
    lambda_payload = {"body": {"text_input": input_list}}

    try:
        invoke_response = lambda_client.invoke(
            FunctionName=lambda_function,
            InvocationType="RequestResponse",
            Payload=js.dumps(lambda_payload),
        )

        lambda_output = (
            invoke_response["Payload"].read().decode("utf8").replace("'", '"')
        )
        response = js.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            embedding_vector = np.asarray(js.loads(response_body)["embeddings"])

        else:
            embedding_vector = np.asarray(js.loads(response_body)["embeddings"])

        return embedding_vector

    except Exception as e:
        print(e)
        embedding_vector = np.zeros((1, 512))
        return embedding_vector

In [ ]:
vec1 = get_embeddings(input_list=["AWS lambda function"])
vec2 = get_embeddings(input_list=["AWS serverless computing"])

In [ ]:
sim = 1 - cosine(vec1, vec2)

In [ ]:
sim

In [ ]:
def hash_func(vecs, projections):
    bools = np.dot(vecs, projections.T) > 0
    return [bool2int(bool_vec) for bool_vec in bools]

def bool2int(x):
    y = 0
    for i,j in enumerate(x):
        if j: y += 1<<i
    return y

In [ ]:
class Table:
    
    def __init__(self, hash_size, dim):
        self.table = dict()
        self.hash_size = hash_size
        self.projections = np.random.randn(self.hash_size, dim)

    def add(self, vecs, label):
        entry = {'label': label}
        hashes = hash_func(vecs, self.projections)
        for h in hashes:
            if h in self.table.keys():
                self.table[h].append(entry)
            else:
                self.table[h] = [entry]

    def query(self, vecs):
        hashes = hash_func(vecs, self.projections)
        results = list()
        for h in hashes:
            if h in self.table.keys():
                results.extend(self.table[h])
        return results

class LSH:
    
    def __init__(self, dim):
        self.num_tables = 4
        self.hash_size = 8
        self.tables = list()
        for i in range(self.num_tables):
            self.tables.append(Table(self.hash_size, dim))
    
    def add(self, vecs, label):
        for table in self.tables:
            table.add(vecs, label)
    
    def query(self, vecs):
        results = list()
        for table in self.tables:
            results.extend(table.query(vecs))
        return results

    def describe(self):
        for table in self.tables:
            print(table.table)

class Search:
    
    def __init__(self, input_dict):
        self.dim_size = 512
        self.lsh = LSH(self.dim_size)
        self.input = input_dict
        self.num_features_in_input = dict()
        for f, kw in self.input.items():
            self.num_features_in_input[f] = 0
                
    def featurize(self):
        for user, kw_list in self.input.items():
            kw_features = get_embeddings(input_list=kw_list)
            self.lsh.add(kw_features, user)
            self.num_features_in_input[user] += len(kw_features)
                
    def query(self, user_dict, user):
        kw_list = user_dict[user]
        kw_features = get_embeddings(input_list=kw_list)
        
        results = self.lsh.query(kw_features)
        print('num results', len(results))

        counts = dict()
        for r in results:
            if r['label'] in counts.keys():
                counts[r['label']] += 1
            else:
                counts[r['label']] = 1
        for k in counts:
            counts[k] = float(counts[k])/self.num_features_in_input[k]
        return counts

In [ ]:
ws = Search(input_dict=ref_user_kw_dict)

In [ ]:
ws.featurize()

In [ ]:
ws.num_features_in_input

In [ ]:
result = ws.query(user_dict=user_kw_dict, user='e034c7a6-0681-40ba-b7b2-b78de2a2cc3d')

In [ ]:
for r in sorted(result, key=result.get, reverse=True):
    print(r, ref_user_id_dict[r], result[r])

In [ ]:
ref_user_kw_dict['d5934fd1-809a-47d9-9ed8-fa81b22f3657']

In [ ]:
user_kw_dict['e034c7a6-0681-40ba-b7b2-b78de2a2cc3d']